In [ ]:
import syft_client as sc
import os, json


# Credentials

In [ ]:
email_do = os.environ["BEACH_EMAIL_DO"]
email_ds = os.environ["BEACH_EMAIL_DS"]

# syft-client/credentials/token_<do/ds>.json
token_path_do = sc.CREDENTIALS_DIR / "token_do.json"
token_path_ds = sc.CREDENTIALS_DIR / "token_ds.json"

# Reset Syftboxes

In [ ]:
def remove_syftboxes_from_drive(email_do, email_ds, token_path_do, token_path_ds):
    from syft_client.sync.syftbox_manager import SyftboxManager
    manager_ds, manager_do = SyftboxManager.pair_with_google_drive_testing_connection(
        do_email=email_do,
        ds_email=email_ds,
        do_token_path=token_path_do,
        ds_token_path=token_path_ds,
        add_peers=False,
    )
    manager_ds.delete_syftbox()
    manager_do.delete_syftbox()

In [ ]:
!rm -rf /Users/koen/SyftBox_koenlennartvanderveen@gmail.com
!rm -rf /Users/koen/SyftBox_koen@openmined.org

In [ ]:
remove_syftboxes_from_drive(email_do, email_ds, token_path_do, token_path_ds)

# Login

In [ ]:
client_ds = sc.login(
    email=email_ds,
    token_path=token_path_ds,
)
client_ds.peers


In [ ]:
client_do = sc.login_do(
    email=email_do,
    token_path=token_path_do,
)

In [ ]:
client_ds.add_peer(email_do)

# Create Dataset 

In [ ]:
MOCK_DATASET_URL = "https://raw.githubusercontent.com/OpenMined/datasets/refs/heads/main/beach/sales-dataset/mock/sales.csv"
PRIVATE_DATASET_URL = "https://raw.githubusercontent.com/OpenMined/datasets/refs/heads/main/beach/sales-dataset/private/sales.csv"
mock_path = "sales_mock.csv"
private_path = "sales_private.csv"
readme_path = "readme.md"
!curl -o $mock_path $MOCK_DATASET_URL
!curl -o $private_path $PRIVATE_DATASET_URL
!echo "# Dataset Readme\nThis is a readme file for the dataset." > ./readme.md


In [ ]:
client_do.create_dataset(
    name="my dataset",
    mock_path=mock_path,
    private_path=private_path,
    summary="This is a summary",
    readme_path=readme_path,
    tags=["tag1", "tag2"],
)

In [ ]:
client_do.datasets.get_all()

In [ ]:
client_do.sync()

In [ ]:
client_ds.sync()

In [ ]:
client_ds.datasets.get_all()

# Submit Job

In [ ]:
%%writefile /tmp/test.py
import os
import json

os.mkdir("outputs")
with open("outputs/result.json", "w") as f:
    json.dump({"result": 1}, f)

In [ ]:
client_ds.submit_python_job(
    user=email_do,
    code_path="/tmp/test.py",
    job_name="test1.job",
)


In [ ]:
client_do.sync()

# Run job and share result

In [ ]:
assert len(client_do.job_client.jobs) > 0
job = client_do.job_client.jobs[0]

job.approve()

client_do.job_runner.process_approved_jobs()

client_do.sync()


In [ ]:
client_ds.sync()

output_path = client_ds.job_client.jobs[-1].output_paths[0]
with open(output_path, "r") as f:
    json_content = json.loads(f.read())

assert json_content["result"] == 1
print(json_content)